In [1]:
import numpy as np
import sys
import math
import itertools
import matplotlib.pyplot as plt  
from time import time 

# Algoritmos avaros

## 5.1 Introducción

En esta unidad nos dedicaremos casi enteramente a resolver problemas de optimización utilizando, como venimos haciendo, algoritmos y no heurísticas. Hacemos énfasis en este punto ya que la estrategia que veremos será reutilizada más adelante para diseñar heurísticas. Pero en lo que respecta a esta unidad, no nos alcanza con encontrar la solución que nos de un costo bajo ---o _score_ alto---, sino que buscamos la solución óptima, la que nos da el costo más bajo posible, o el _score_ más alto.

Los algoritmos _greedy_, o golosos, o avaros, pueden ser entendidos como un subtipo de los algoritmos de PD.
Resuelven problemas que pueden ser resueltos por algoritmos de PD, pero siguiendo una estrategia más directa lo que resulta en un algoritmo con una menor complejidad.
Los problemas que pueden ser resueltos por algoritmos golosos, como los que se resuelven por PD, tienen subestructura óptima. Pero a diferencia de PD, cada subproblema depende de la solución de un solo subproblema más pequeño. Esta crucial diferencia elimina la multiplicidad de caminos posibles, aplanando los árboles de recursión de PD y conviertiéndolos en un proceso lineal en el que se decide el camino óptimo de recursión a cada paso; a diferencia de PD que debe explorar distintas posibilidades para luego decidir el camino óptimo.

<img src="arbol_pd_greedy.png" alt="mm" style="width: 1000px;"/>

Árboles de subproblemas de un algoritmo de PD y uno _greedy_ para un problema hipótetico que tiene subestructura óptima y además puede ser resuelto por método _greedy_. 
Tal subproblema hipotético podría ser resuelto por PD, pero si se opera de la forma correcta, puede ser resuelto en forma directa, linealizando subproblemas y eliminando la superposición de los mismos.
En _greedy_ el camino óptimo para resolver un subproblema es evidente, esto lleva a un camino directo que lleva el problema inicial hasta el caso base.

<img src="ejer3.png" alt="mm" style="width: 600px;"/>

El ejercicio 3 de la guía de PD puede ayudar con la diferenciación. A la izquierda está la matriz de costos de la que debiamos partir desde `[0,0]` y llegar hasta `[n,m]` y a la derecha, la estructura de datos final que nuestro algoritmo de PD llenó con los costos de las soluciones óptimas a los subproblemas.
Luego de terminar nuestro algoritmo que hallaba estos costos, debíamos hallar la solución óptima en sí. Esto lo podíamos hacer tomando la estructura de datos donde memoizamos y haciendo unn _traceback_ desde la celda `[n,m]` hasta la celda `[0,0]` (por cierto, este procedimiento es muy similar al _traceback_ de Needleman-Wunsch).

Este algoritmo final de _traceback_ es un algoritmo greedy. Parte desde la celda `[n,m]`, que tiene un valor de `18`, se fija cuál es la celda contigua con menor costo y avanza hacia ella, sin plantearse un camino alternativo y aún así, halla la respuesta correcta. Esto es común a todos los algoritmos _greedy_. Éstos hacen la elección óptima local, toman la decisión que parece más conveniente en el momento y aún así llegan a la respuesta óptima a nivel global.

Ésto, claro, se debe a que PD ya hizo su trabajo y en vez de lidiar con una matriz de costos de cada celda, el algoritmo _greedy_ está lidiando con los costos **para llegar** a cada celda.

Si utilizaramos el algoritmo _greedy_ para la primer parte del problema, no tendríamos garantizada la respuesta correcta. En cambio, podríamos usar un algoritmo de PD para reemplazar al _greedy_ en la segunda parte del problema y si bien obtendríamos la respuesta correcta, sería con un exceso de cálculo; con un algoritmo _greedy_ hubiera bastado.

Pero es importante notar que las aproximaciones lidian con problemas que un análisis superficial tacharía de similares, pero que ahora los sabemos fundamentalmente distintos.

-------

## 5.2 _Stable Matching_

El ejemplo que veremos será el de un problema de decisión y no de optimización. 

En una realidad alternativa, las incorporaciones de los jugadores a los clubes se maneja de la siguiente manera:
\
hay `n` clubes ofreciendo una vacante para incorporador un jugador, de `m` disponibles. Cada club tiene un orden de preferencia de jugadores y a su vez, cada jugador tiene un orden de preferencia de clubes para jugar. Si se dan las siguientes condiciones:
 
 * Mismo número de clubes y jugadores (`n == m`).
 * Cada club ofrece 1 vacante.
 * Los órdenes de preferencia son completos, es decir, cada jugador ranquea a todos los clubes y lo mismo hace cada club con los jugadores.
 * No hay empates entre los rankings de clubes y jugadores
 
entonces existe un conjunto de apareamientos completo que no sea inestable.

**Apareamiento inestable**: ocurre cuando hay un par club-jugador que preferirían estar en la misma pareja entre sí, por sobre su pareja actual. Es decir, una vez hecho el apareamiento de todos los clubes con todos los jugadores, no debería ser posible hallar 2 pares club-jugador en donde uno de los jugadores prefiera jugar en el otro club y este otro club también prefiera a ese jugador, por sobre el que se le dió.

Nótese que este no es un problema de optimización. No buscamos maximizar la satisfacción de los clubes, ni de los jugadores, ni de ambos grupos en conjunto. Buscamos un apareamiento que sea estable, según la definición anterior. Además el apareamiento debe ser completo, no debe quedar jugador sin club, ni club sin jugador.

Siempre ayuda tener un ejemplo reducido. Las letras en mayúsculas son los clubes y las minúsculas, los jugadores. Las 2 tablas presentan sus órdenes de preferencia.

<img src="stable_matching_1.png" alt="mm" style="width: 300px;"/>

------------

## 5.3 Posible solución al _Stable Matching_

Anteriormente dijimos que un algoritmo _greedy_ toma la decisión más óptima posible a medida que se le presentan las instancias de decisión. Si bien aquí no hay solución óptima, por que no hay nada optimizar, porque éste no es un problema de optimización, si hay que tomar decisiones. 

Así que eso haremos, a la hora de aparear `n` clubes y `m` jugadores, no nos plantearemos las $nm$ posibles parejas, sino que le daremos a cada club que se nos presente un jugador y pasaremos al próximo club y jugador (subproblema). También se podría hacer desde la perspectiva de los jugadores, pero si logramos un algoritmo correcto (que no genere parejas inestables en ningún caso), esa inversión de perspectiva no será relevante.

Comenzaremos iterando a lo largo de los clubes y dándole a cada club, el jugador de su preferencia, mientras éste esté disponible. Obtenemos el siguiente apareamiento:

$$
\{A ; r\} \newline
\{B ; s\} \newline
\{C ; q\} \newline
$$

Y así no hay apareamientos inestables, ya que cada club se quedó con su jugador preferido. Veamos que ocurre si invertimos la perspectiva

$$
\{q ; A\} \newline
\{r ; C\} \newline
\{s ; B\} \newline
$$

Tampoco es inestable.

Veamos un nuevo ejemplo:

<img src="stable_matching_2.png" alt="mm" style="width: 300px;"/>

Apareamos desde la perspectiva de los clubes:

$$
\{A ; q\} \newline
\{B ; r\} \newline
\{C ; s\} \newline
$$

Y aquí se rompe nuestra solución. $B$ hubiera preferido a $q$, que a su vez lo hubiera preferido antes que quedarse con $A$, el último club de su lista. Este apareamiento es inestable.


------------

## 5.4 Algoritmo de Boston Pool y Gale-Shapley

Nuestro problema es que no fuimos suficientemente _greedy_. Tomamos la decisión avara para los clubes, pero no para los jugadores.

Un error común es suponer que porque un algoritmo es avaro y resuelve un subproblema de inmediato, no debe ser capaz de corregir esa decisión en un paso posterior.

El algoritmo avaro correcto itera los clubes como el anterior, pero si el jugador preferido ya tiene club, no pasa al siguiente jugador, sino que le pregunta al jugador si no preferiría descartar a su club anterior y formar una nueva pareja, ya que hay un club que lo prefiere a él. Y éste segundo paso es la clave para que no se formen parejas inestables.\
Naturalmente, esto deja a clubes rechazados sin jugadores, el algoritmo debe volver a iterar por los clubes restantes hasta que no haya club sin jugador y esto garantiza que no haya club ni jugador sin aparear.

Entonces, el procedimiento sería:

 * Un club $A$ se le ofrece al jugador $\alpha$ de su máxima preferencia, que no lo haya rechazado aún.
 * Ahora se pueden dar 3 casos:

```
1. El jugador α no tiene club y acepta tentativamente la oferta.
2. El jugador α tiene club, pero prefiere a A. Acepta tentativamente la oferta.
3. El jugador α tiene club, y lo prefiere a A. Rechaza la oferta de forma definitoria.
```

Es decir, los clubes ofertan de forma avara y los jugadores aceptan de forma avara. Y de nuevo, si invertimos los roles, nada va a cambiar. 

Si aplicamos este nuevo algoritmo al ejemplo anterior, obtenemos el apareamiento:

Primera pasada:

$$
\{A ; \} \newline
\{B ; q\} \newline
\{C ; s\} \newline
$$

Segunda pasada:
$$
\{A ; s\} \newline
\{B ; q\} \newline
\{C ; \} \newline
$$

Tercera pasada:
$$
\{A ; s\} \newline
\{B ; q\} \newline
\{C ; r\} \newline
$$

Pueden ustedes practicar con este nuevo ejemplo:

<img src="stable_matching_3.png" alt="mm" style="width: 300px;"/>

------------

## 5.5 Corrección y complejidad del algoritmo de Boston Pool y Gale-Shapley

Es fundamental en los algoritmos avaros comprobar que nuestra solución llegue a una solución correcta, ya que no todo problema puede ser resuelto óptima o correctamente, por un algoritmo _greedy_ e incluso aquellos que si lo son, deben ser resueltos con la estrategia correcta. Ya vimos que la estrategia errónea puede llevar a una solución incorrecta, o en el caso de los problemas de optimización, una solución no óptima.

Comprobaremos las 2 propiedades que nuestra solución debe tener.

* Bajo este algoritmo, cada club hace $1 \leq ofertas \leq m$ ofertas y se termina con un apareamiento completo. Si no lo fuera, entonces habría un club que fue rechazado por todos los jugadores y para que eso ocurra, entonces los `m` jugadores deberían tener club, pero como `m==n`, entonces los `n` clubes también tendrían su jugador y por lo tanto no podría haber un club sin jugador (contradicción, absurdo).
* El apareamiento es estable. Dados un club y un jugador `{u, v}` que no fueron apareados, estos no preferirían estar juntos ya que si no están apareados se dió alguno de estos 2 casos:

```
1. el club u nunca ofertó al jugador v, en tal caso, el club tiene un jugador w al que prefiere por encima de v.
2. el club u hizo una oferta al jugador v, pero éste prefirió a un club x
```
Entonces no hay apareamiento inestable.

Y en la primera comprobación vimos un indicio de la complejidad de nuestro algoritmo. Si `m == n` y cada club oferta como máximo a todos los jugadores, y a cada paso hacemos una oferta, entonces el número de ofertas y, por lo tanto, la complejidad de nuestro algoritmo estarán acotados por $O(n)$

------------

## 5.6 Implementación del algoritmo de Boston Pool y Gale-Shapley

In [93]:
def estable(orden_clubes, orden_jugadores):
    n = len(orden_jugadores)
    apareamiento = {"q": None, "r": None, "s": None, "t": None}
    ocupacion_clubes = {"A": False, "B": False, "C": False, "D": False}
    
    while(True):
        nro_clubes_ocupados = sum([ value for key,value in ocupacion_clubes.items() ])
        if nro_clubes_ocupados == n:
            break
        for club,preferencias in orden_clubes.items():
            if ocupacion_clubes[club] == False:
                for jugador in preferencias:
                    club_viejo = apareamiento[jugador]

                    if club_viejo == None:
                        apareamiento[jugador] = club
                        ocupacion_clubes[club] = True
                        break
                    else:
                        if (orden_jugadores[jugador][club] < orden_jugadores[jugador][club_viejo]):
                            apareamiento[jugador] = club
                            ocupacion_clubes[club] = True
                            ocupacion_clubes[club_viejo] = False
                            break
    return apareamiento

In [95]:
orden_jugadores = {"q": {"A": 0, "B": 1, "C": 2, "D": 3},
"r": {"A": 0, "B": 3, "C": 2, "D": 1},
"s": {"A": 1, "B": 0, "C": 2, "D": 3},
"t": {"A": 3, "B": 2, "C": 1, "D": 0}}

orden_clubes = {"A": ['t', 's', 'r', 'q'],
                "B": ['r', 't', 'q', 's'],
                "C": ['t', 'r', 's', 'q'],
                "D": ['s', 'r', 'q', 't']}

estable(orden_clubes, orden_jugadores)

{'q': 'B', 'r': 'D', 's': 'A', 't': 'C'}

------------

## 5.7 Características de un problema que puede ser resuelto por un algoritmo _greedy_

Como ya se dijo, un algoritmo _greedy_ la solución óptima a un problema tomando decisiones en serie.
A cada instancia de decisión, el algoritmo toma la decisión que parece mejor en el momento.
Para que esta estrategia lleve a una respuesta óptima, el problema a resolver debe tener 2 características:
1. Propiedad _greedy_
2. Subestructura óptima

### Propiedad _greedy_

Para poder resolver mediante un algoritmo avaro, debemos podemos ensamblar una solución óptima tomando decisiones localmente óptimas (avaras).
En otras palabras, cuando estamos considerando qué elección tomar, tomamos la decisión que se ve mejor en el problema actual, sin considerar los resultados de los subproblemas subsiguientes.

Aquí es donde los algoritmos avaros se diferencian de los de PD.
En PD, la elección depende de las soluciones a los subproblemas más pequeños, hasta el caso base (digamos, `n == 0`).
En consecuencia, resolvemos en primer lugar a los subproblemas más pequeños y en base a esas soluciones,
decidimos como resolver los subproblemas más grandes hasta llegar al problema original (llamémosle, `n == n`).
Independientemente de que el algoritmo sea _bottom-up_ o _top-down_.

En un algoritmo avaro, hacemos la elección que parece la mejor en ese momento y luego resolvemos el subproblema que queda. No hay subproblemas alternativos porque ya tomamos una decisión.
La elección hecha por un algoritmo avaro puede depender de elecciones anteriores, pero no puede depender de las opciones futuras, de las soluciones a los subproblemas futuros. 
Por eso los algoritmos _greedy_ suelen seguir una estrategia _top-down_, ocupandose del subproblema `n`, sin preocuparse de los más pequeños o un caso base, hasta que llegue el momento de resolverlos.


Por supuesto, para poder hacer esto, debemos demostrar que una elección avara en cada paso produce una solucion óptima global.
Y para que esto suceda, el problema debe tener, además, subestructura óptima.

### subestructura óptima

Un problema exhibe una subestructura óptima si una solución óptima al problema contiene en su interior soluciones óptimas a subproblemas.
Esta propiedad es necesaria para aplicar algoritmos de PD (como ya vimos), y para los avaros también.
Es la propiedad que permite aplicar un mismo algoritmo a problemas y subproblemas y en la que que se basa la comprobación de nuestro algoritmo
para inferir, inductivamente, que la solución óptima de un subproblema, 
lleva a otro subproblema cuya solución óptima también pueda ser hallada por el método _greedy_ y así concluir que el problema original (con todos sus subproblemas) puede ser resuelto por esta estrategia.

## 5.8 Pasos en el diseño de un algoritmo _greedy_

* Estructurar el problema de modo tal que se resuelva descomponiéndolo en subproblemas, que pueden ser resueltos tomando una única decisión y que eso nos deje con un sólo subsiguiente subproblema a resolver. Este paso suele tener consecuencias directas en el pseudocódigo final. Los algoritmos avaros suelen incluir una preparación previa de su entrata para poder aplicar el algoritmo _greedy_ de forma correcta.
* Probar que la solución óptima al problema global puede ser hallada tomando una decisión inmediata.
* Demostrar la subestructura óptima mostrando que, habiendo tomado un único camino, lo que queda es un subproblema que al ser resuelto con la misma estrategia, termina llevando a la solución óptima del problema original.

### Sobre la comprobación

En nuestro ejemplo llevamos a cabo una comprobación de la corrección de nuestro algoritmo utilizando el método del absurdo.
Éste es un método útil, pero usualmente los algoritmos avaros se comprueban siguiendo un esquema de intercambio inductivo:

1. Se supone que existe una solución óptima diferente a la solución avara.
2. Se toma un elemento de cada conjunto solución y se supone que  son diferentes.
3. Se argumenta que se puede intercambiar el elemento de la solución óptima por el elemento de la solución avara sin empeorar la solución óptima.
4. Si esto se continua, se termina con una solución óptima que en realidad es la avara (paso inductivo).

----------------

## 4.8 Conclusión

* **subestructura óptima**
* **propiedad _greedy_**
* advertencia: el código de un algoritmo _greedy_ puede ser simple. El desafío está en saber cuando aplicar esta estrategia, poder presentar el problema de modo tal que un algoritmo _greedy_ lo resuelva y luego comprobar que tal algoritmo realmente lleva a una solución óptima. Es decir, casi todo el trabajo está en la etapa previa a escribir el pseudocódigo.
* comprobar la corrección del algoritmo avaro
* Leer del Cormen ; Chapter 16: Greedy Algorithms: p(414-436). Hay un buen apartado de PD Vs. Greedy para los problemas de mochila binario y fraccionario.

--------------------

### Contenidos a explicar durante la práctica

1. Problema de maximación de la ocupación un único recurso